## LAB<sup>5-1</sup> 붓꽃 데이터를 군집화하도록 하자

**실습목표**

iris 데이터에 대하여 sklearn에서 제공하는 k-평균 군집화 알고리즘을 적용하자. 이때 target 정보는 이용하지 말고, 4개의 특성값만을 이용하여 3개의 군집으로 나누어 보도록 한다. 다음으로 k-평균 군집화 알고리즘을 이용하여 군집화된 데이터의 레이블 정보를 출력하면 다음 출력과 같이 [1, 1, 1, ..., 0, 0, 0, ..., 2, 2, 2, ..]로 레이블링 될 수 있다. 이와 같이 출력되는 이유는 군집화 알고리즘에서는 각각의 군집에 대한 레이블만을 출력할 뿐 어느 군집이 setosa(0), versicolor(1), virginica(2)에 속하는지에 대한 target 정보가 없기 때문이다. 이제 이 정보를 바탕으로 다시 레이블링을 하여 new_label을 만들어 보자. 마지막으로, 이 군집화 결과값과 원래 iris 데이터의 target과의 차이를 비교하여 정확도를 다음과 같이 출력하도록 하자.


**원하는 결과**
```
kMeans() 군집화 적용...
군집화 결과 labels = [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...2 2 2 2 0 2 2 2 0 2 2 2 0 2 2 0]
다시 레이블링을 한 후의 new_labels = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...2 2 2 2 1 2 2 2 1 2 2 2 1 2 2 1]
iris 데이터의 군집화 정확도: 0.8933333333333333
```

1. 우선 Means() 함수를 사용하기 위하여 cluster 모듈을 가져온다. sklearn의 datasets 모듈의 load_iris() 함수를 임포트하고, iris 데이터를 읽어 온다. 그리고 이 데이터를 몇 개의 군집으로 나눌지 k 값을 결정하도록 한다.

In [1]:
from sklearn import cluster
from sklearn.datasets import load_iris
import numpy as np

iris = load_iris()
k = len(iris.target_names) # 몇 개의 군집으로 나눌지를 결정한다.

2. cluster 모듈의 KMeans() 함수를 호출하고 3개의 그룹으로 나눌 수 있도록 model을 만든다. 이제 iris.data를 입력으로 주어 모델을 최적화시킨 다음 이 모델에 iris.data를 입력으로 주고 labels를 얻도록 한다. 이 레이블 데이터는 클러스터링된 데이터에 대한 레이블로 iris.target 값과 같지 않을 수도 있다. 따라서 labels를 출력한 후 1/3 등분하여 넘파이의 bincount() 함수에 넣도록 한다. bincount() 함수는 객체의 원소중 0부터 최대값 범위의 정수값을 오름차순으로 정리한 뒤 각 원소에 대한 빈도수를 반환한다. 따라서 이 함수는 0, 1, 2 값의 빈도수를 구하는데 사용될 수 있다 한다. 예를 들어 [2, 2, 2, 4, 4]와 같은 값이 있을 경우 bincount 함수를 적용하면 [0, 0, 3, 0, 2]와 같은 출력이 나온다.

In [2]:
print('kMeans() 군집화 적용...')
model = cluster.KMeans(n_clusters=k)
model.fit(iris.data)
labels = model.predict(iris.data)
print('군집화 결과 labels :', labels)
a_counts = np.bincount(labels[:50])
b_counts = np.bincount(labels[50:100])
c_counts = np.bincount(labels[100:])

kMeans() 군집화 적용...
군집화 결과 labels : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 2 2 1 2 2 2 2
 2 2 1 1 2 2 2 2 1 2 1 2 1 2 2 1 1 2 2 2 2 2 1 2 2 2 2 1 2 2 2 1 2 2 2 1 2
 2 1]


3. 이제 np.argmax(a_counts)를 이용하여 a_count 배열에서 가장 자주 출현하는 값을 0에 대응시키고, b_count 배열에서 가장 자주 출현하는 값을 1에 대응시키는 방식으로 1:1 대응값을 가지는 딕셔너리 d를 만들도록 하자. 이제 labels의 값을 new_labels에 복사한 후 for 문을 실행시키자. 이때 d의 키<sup>key</sup>와 값<sup>value</sup>을 각각 old, new에 담자. 그러면 이제 new_labels[ labels == old ]와 같이 불리언 인덱싱을 통해 기존 labels의 old 값을 new 값으로 대치할 수 있다.

In [3]:
d= {0: np.argmax(a_counts), 1: np.argmax(b_counts), 2: np.argmax(c_counts)}
new_labels = np.copy(labels)
for old, new in d.items():
  new_labels[labels == old] = new

print('다시 레이블링을 한 후의 new_labels =', new_labels)

다시 레이블링을 한 후의 new_labels = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 2 2 1 2 2 2 2
 2 2 1 1 2 2 2 2 1 2 1 2 1 2 2 1 1 2 2 2 2 2 1 2 2 2 2 1 2 2 2 1 2 2 2 1 2
 2 1]


4. 마지막으로 새로운 new_labels와 iris.target의 값을 비교하여 정확도를 출력하자.

In [4]:
from sklearn.metrics import accuracy_score
print('iris 데이터의 군집화 정확도:', accuracy_score(iris.target, new_labels))

iris 데이터의 군집화 정확도: 0.8933333333333333
